In [ ]:
import pandas as pd

df = pd.read_csv(r'Customer-Churn-Records.csv')
df

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv(r'Customer-Churn-Records.csv')


#print(df_encoded.columns)
y = df['Exited'].astype(int)
#df_encoded = df.astype(float)
#df_encoded
#X=df_encoded.drop(['Churn'], axis=1)

df= df[['CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'EstimatedSalary',
       'Exited', 'Satisfaction Score',
       'Point Earned']]

df = pd.DataFrame(df)

# 初始化LabelEncoder
label_encoder = LabelEncoder()


    
X=df.drop(['Exited'], axis=1)

In [ ]:
import scipy.stats as stats

# Shapiro-Wilk test
for column in df.columns:
    _, p_value  = stats.shapiro(df[column])
    if p_value > 0.05:
        print(f"{column} seems to be normally distributed (p={p_value:.2f})")
    else:
        print(f"{column} does not seem to be normally distributed (p={p_value:.2f})")

In [ ]:
#RF baseline in the paper


import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, matthews_corrcoef, confusion_matrix, classification_report)

from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt



# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Summarize class distribution
print("Before SMOTE: ", dict(zip(*np.unique(y_train, return_counts=True))))

# Apply SMOTE to generate synthetic samples
smote = SMOTE(random_state=1)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Summarize the new class distribution
print("After SMOTE: ", dict(zip(*np.unique(y_resampled, return_counts=True))))

# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the model on the resampled dataset
rf_classifier.fit(X_resampled, y_resampled)

# Predict on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the classifier

y_prob = rf_classifier.predict_proba(X_test)[:, 1]


auc_score = roc_auc_score(y_test, y_prob)

# 評估模型
print("AUC:", auc_score)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Matthews Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, matthews_corrcoef, confusion_matrix, classification_report)


from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale_pos_weight
scale_pos_weight = sum(y_train == 0) / sum(y_train == 1)

# parameter
param_grid = {
    'objective': ['binary:logistic'],
    'scale_pos_weight': [scale_pos_weight],
    'max_depth': [3,4,5],
    'learning_rate': [0.1,0.01,0.2],
    'n_estimators': [300,400,500],
    'verbosity': [0]  # 使用verbosity取代過時的silent參數
}

model = xgb.XGBClassifier()

model.fit(X_train, y_train)



# 
y_pred = model.predict(X_test)

# 
y_prob = model.predict_proba(X_test)[:,1]


auc_score = roc_auc_score(y_test, y_prob)

# 
print("AUC:", auc_score)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Matthews Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
# Feature importance

booster = model.get_booster()
feature_importance = booster.get_score(importance_type='cover')

# 
sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
feature_names, feature_scores = zip(*sorted_features)

# 
plt.figure(figsize=(12, 8))
plt.barh(feature_names, feature_scores)
plt.xlabel('Score')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.gca().invert_yaxis()  # 這行是為了將最重要的特徵放在頂部
plt.show()
feature_names,feature_scores

In [ ]:
import matplotlib.pyplot as plt

# 
booster = model.get_booster()
feature_importance = booster.get_score(importance_type='weight')

# 
sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
feature_names, feature_scores = zip(*sorted_features)

# 
plt.figure(figsize=(12, 8))
plt.barh(feature_names, feature_scores)
plt.xlabel('Score')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.gca().invert_yaxis() 
plt.show()

feature_names, feature_scores

In [ ]:
import numpy as np
import shap
explainer = shap.Explainer(model)
shap_values = explainer.shap_values(X_train)


#Get shap value summary
shap_values
result = [np.mean([abs(items[i]) for items in shap_values]) for i in range(len(shap_values[0]))]

#Get column name
X.columns
result
# DataFrame
df = pd.DataFrame({'Feature': X.columns, 'mean|shap value|': result})
sorted_df = df.sort_values(by='mean|shap value|',ascending=False)
# DataFrame
sorted_df

In [ ]:
import numpy as np
import shap
import pandas as pd

# ... (your code to train the model, etc.)

# Assuming 'model' is your trained model
explainer = shap.Explainer(model)
shap_values = explainer.shap_values(X_train)

# Plotting the SHAP values with a beeswarm plot
shap.summary_plot(shap_values, X_train,max_display=10,color="coolwarm",)

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv(r'Customer-Churn-Records.csv')


#print(df_encoded.columns)
y = df['Exited'].astype(int)
#df_encoded = df.astype(float)
#df_encoded
#X=df_encoded.drop(['Churn'], axis=1)

df= df[['CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'IsActiveMember', 'EstimatedSalary',
       'Exited', 'Satisfaction Score', 
       'Point Earned']]

df = pd.DataFrame(df)

# LabelEncoder
label_encoder = LabelEncoder()

    
X=df.drop(['Exited'], axis=1)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=40)
X_temp=pd.concat([X_temp,y_temp],axis=1)
X=X_temp[X_temp['Exited']==1]

X

In [ ]:
X.shape

In [ ]:
X.dtypes

Xtofuzzy_Tri  = X[['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary','Satisfaction Score','Point Earned','IsActiveMember']]

## Fuzzy Triangle & Gaussion



In [ ]:
import pandas as pd

# Membership function
def triangular_mf(x, a, b, c):
    if x <= a:
        return 0
    elif a < x <= b:
        return (x - a) / (b - a)
    elif b < x <= c:
        return (c - x) / (c - b)
    else:
        return 0

# Maximal cardinality
def fuzzy_classification(x, min_val, median_val, max_val):
    low_membership = triangular_mf(x, min_val, min_val, median_val)
    medium_membership = triangular_mf(x, min_val, median_val, max_val)
    high_membership = triangular_mf(x, median_val, max_val, max_val)
    
    max_membership = max(low_membership, medium_membership, high_membership)
    
    if max_membership == low_membership:
        return 0
    elif max_membership == medium_membership:
        return 1
    else:
        return 2
  
    
fuzzy_df = pd.DataFrame()
# For each feature
for feature in Xtofuzzy_Tri.columns:
    min_val = Xtofuzzy_Tri[feature].min()
    median_val = Xtofuzzy_Tri[feature].quantile(0.5)
    max_val = Xtofuzzy_Tri[feature].max()
    
    fuzzy_df[f'{feature}_fuzzy'] = Xtofuzzy_Tri[feature].apply(lambda x: fuzzy_classification(x, min_val, median_val, max_val))

fuzzy_df

# Define a dict
boundaries = {}

for feature in Xtofuzzy_Tri.columns:
    min_val = Xtofuzzy_Tri[feature].min()
    median_val = Xtofuzzy_Tri[feature].quantile(0.5)
    max_val = Xtofuzzy_Tri[feature].max()
    
    # Save
    boundaries[feature] = (min_val, median_val, max_val)

# print out
for feature, bounds in boundaries.items():
    print(f"Feature: {feature}")
    print(f"Low boundary: {bounds[0]}, Medium boundary: {bounds[1]}, High boundary: {bounds[2]}")
    print("------")

fuzzy_df

In [ ]:
import pandas as pd
import numpy as np

# Membership function
def gaussian_mf(x, c, sigma):
    return np.exp(-((x - c) ** 2) / (2 * sigma ** 2))

# 模糊分類函數
def fuzzy_classification_g(x, c_low, c_mid, c_high, sigma):
    low_membership = gaussian_mf(x, c_low, sigma)
    medium_membership = gaussian_mf(x, c_mid, sigma)
    high_membership = gaussian_mf(x, c_high, sigma)
    
    max_membership = max(low_membership, medium_membership, high_membership)
    
    if max_membership == low_membership:
        return 0
    elif max_membership == medium_membership:
        return 1
    else:
        return 2

# 
boundaries = {}
output_df = pd.DataFrame()  

# For each feature
for feature in Xtofuzzy_Gaus.columns:
    c_low = Xtofuzzy_Gaus[feature].quantile(0.25)
    c_mid = Xtofuzzy_Gaus[feature].quantile(0.5)
    c_high = Xtofuzzy_Gaus[feature].quantile(0.75)
    sigma = Xtofuzzy_Gaus[feature].std()
    
    # 儲存分類邊界
    boundaries[feature] = (c_low, c_mid, c_high)
    
    output_df[f'{feature}_fuzzy'] = Xtofuzzy_Gaus[feature].apply(lambda x: fuzzy_classification_g(x, c_low, c_mid, c_high, sigma))

# 
feature_to_retrieve = "TTL DAY MIN"
c_low, c_mid, c_high = boundaries[feature_to_retrieve]
print(f"c_low for TTL DAY MIN: {c_low}")
print(f"c_mid for TTL DAY MIN: {c_mid}")
print(f"c_high for TTL DAY MIN: {c_high}")

output_df

In [ ]:
import pandas as pd

fuzzy_columns = [col for col in df_encoded.columns]

# one-hot encoding
df_encoded = pd.get_dummies(df_encoded, columns=fuzzy_columns)

# 
for col in fuzzy_columns:
    mappings = {
        f"{col}_{i}": f"{col}_{i}" for i in range(3)
    }
    df_encoded.rename(columns=mappings, inplace=True)
df_encoded.columns

In [ ]:
# Initialize dictionary to store dataset
dataset = {}

# Convert to transaction dataset
transaction_id = 1
for _, row in df_encoded.iterrows():
    transaction = {}
    for column, value in row.items():
        transaction[column] = value
    transaction_key = f'transaction{transaction_id}'
    dataset[transaction_key] = transaction
    transaction_id += 1

In [ ]:
transactions = dataset

filtered_transactions = {transaction: {key: value for key, value in items.items() if value != 0}
                        for transaction, items in transactions.items()}

filtered_transactions

## High Utility fuzzy Churn Patterns

In [ ]:
def extract_base_name(fuzzy_name):
    if fuzzy_name.startswith("Status_"):
        return "Status"
    elif fuzzy_name.startswith("Age Group_"):
        return "Age Group"
    elif fuzzy_name.startswith("Tariff Plan_"):
        return "Tariff Plan"
    elif "fuzzy" in fuzzy_name:
        # Splitting at the first underscore and taking the first part
        return fuzzy_name.split("_", 1)[0]
    else:
        return fuzzy_name

def calculate_utility(itemset, transactions, profit_table):
    """
    计算项目集的效用总和
    """
    total_utility = 0
    for transaction in transactions.values():
        # 检查项目集是否包含在交易中
        if set(itemset).issubset(set(transaction.keys())):
            # 加总项目的价值
            for item, utility in transaction.items():
                if item in itemset:
                    # 使用基本名称从profit_table查询利润
                    base_name = extract_base_name(item)
                    total_utility += utility * profit_table[base_name]
    return total_utility

def find_top_k_high_utility_itemsets(transactions, profit_table, k):
    """
    找出前K個高效用項目集
    """
    itemsets = set()
    for transaction in transactions.values():
        # 將交易中的每個項目加入項目集
        itemsets.update(transaction.keys())

    # 儲存高效用項目集及其效用總和
    high_utility_itemsets = {}

    for item in itemsets:
        # 計算單一項目的效用總和
        utility = calculate_utility([item], transactions, profit_table)
        if utility > 0:
            high_utility_itemsets[(item,)] = utility

    # 生成更大的項目集
    P_itemsets = {}
    while True:
        temp_itemsets = {}
        for itemset, utility in high_utility_itemsets.items():
            for item in itemsets:
                if item not in itemset:
                    new_itemset = tuple(sorted(list(itemset) + [item]))
                    new_utility = calculate_utility(new_itemset, transactions, profit_table)
                    if new_utility > 0:
                        temp_itemsets[new_itemset] = new_utility
                        P_itemsets[new_itemset] = new_utility

        # 若 temp_itemsets 為空，停止迭代
        if not temp_itemsets:
            break

        # 選取前K個高效用項目集
        sorted_itemsets = sorted(temp_itemsets.items(), key=lambda x: x[1], reverse=True)
        top_k_itemsets = sorted_itemsets[:k]
        high_utility_itemsets = {itemset: utility for itemset, utility in top_k_itemsets}

    return high_utility_itemsets, P_itemsets

In [ ]:
# SHAP
profit_table = {
   "Age": 1.051942,
    "NumOfProducts": 0.944460,
    "Balance": 0.375520,
    "EstimatedSalary": 0.275761,
    "CreditScore": 0.285694,
    "Point Earned": 0.258406,
    "Tenure": 0.162138,
    "Satisfaction Score": 0.143929,
    "HasCrCard": 0.045952,
    "IsActiveMember":0.510511,
    "Card Type":0.088436,
    "Geography":0.277061,
    "Gender":0.293275
}


# 取得前2個高效用項目集
result,P_itemsets= find_top_k_high_utility_itemsets(filtered_transactions, profit_table, 3)

#print(P_itemsets)
sorted_data = sorted(P_itemsets.items(), key=lambda x: x[1], reverse=True)

# Top 10
top_10 = sorted_data[:10]
top_10 = {itemset: value for itemset, value in top_10}
#print(top_10)
print("Top 10 High Utility Itemsets:")
for itemset, utility in top_10.items():
    print(itemset, "-> Utility:", round(utility,0))

## High Utility Length=3

In [ ]:
def find_top_k_high_utility_itemsets(transactions, profit_table, k, desired_length):
    """
    找出前K個高效用項目集
    """
    itemsets = set()
    for transaction in transactions.values():
        # 將交易中的每個項目加入項目集
        itemsets.update(transaction.keys())

    # 儲存高效用項目集及其效用總和
    high_utility_itemsets = {}
    P_itemsets = {}

    for item in itemsets:
        # 計算單一項目的效用總和
        utility = calculate_utility([item], transactions, profit_table)
        if utility > 0:
            high_utility_itemsets[(item,)] = utility

    while True:
        temp_itemsets = {}
        for itemset, utility in high_utility_itemsets.items():
            for item in itemsets:
                if item not in itemset:
                    new_itemset = tuple(sorted(list(itemset) + [item]))
                    new_utility = calculate_utility(new_itemset, transactions, profit_table)
                    if new_utility > 0 and len(new_itemset) <= desired_length:
                        temp_itemsets[new_itemset] = new_utility
                        P_itemsets[new_itemset] = new_utility

        # 若 temp_itemsets 為空，停止迭代
        if not temp_itemsets:
            break

        # 選取前K個高效用項目集
        sorted_itemsets = sorted(temp_itemsets.items(), key=lambda x: x[1], reverse=True)
        top_k_itemsets = sorted_itemsets[:k]
        high_utility_itemsets = {itemset: utility for itemset, utility in top_k_itemsets}

    return high_utility_itemsets, P_itemsets

In [ ]:
# SHAP
profit_table = {
   "Age": 1.051942,
    "NumOfProducts": 0.944460,
    "Balance": 0.375520,
    "EstimatedSalary": 0.275761,
    "CreditScore": 0.285694,
    "Point Earned": 0.258406,
    "Tenure": 0.162138,
    "Satisfaction Score": 0.143929,
    "HasCrCard": 0.045952,
    "IsActiveMember":0.510511,
    "Card Type":0.088436,
    "Geography":0.277061,
    "Gender":0.293275
}

# 取得前2個高效用項目集
result,P_itemsets= find_top_k_high_utility_itemsets(filtered_transactions, profit_table, 3,2)

# 輸出結果
#print(P_itemsets)
# 使用sorted函式將字典按數值降序排序
sorted_data = sorted(P_itemsets.items(), key=lambda x: x[1], reverse=True)

# 取得前5名的資料
top_10 = sorted_data[:10]
top_10 = {itemset: value for itemset, value in top_10}
#print(top_3)
print("Top 10 High Utility Itemsets:")
for itemset, utility in top_10.items():
    print(itemset, "-> Utility:", round(utility,0))

## Validation

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv(r'Customer-Churn-Records.csv')



y = df['Exited'].astype(int)


df= df[['CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'IsActiveMember', 'EstimatedSalary',
       'Exited', 'Satisfaction Score', 
       'Point Earned']]

df = pd.DataFrame(df)

# LabelEncoder
label_encoder = LabelEncoder()   
    
X=df.drop(['Exited'], axis=1)



fuzzy_df = pd.DataFrame()
# Fuzzy classification
for feature in Xtofuzzy_Tri.columns:
    min_val = Xtofuzzy_Tri[feature].min()
    median_val = Xtofuzzy_Tri[feature].quantile(0.5)
    max_val = Xtofuzzy_Tri[feature].max()
    
    fuzzy_df[f'{feature}_fuzzy'] = Xtofuzzy_Tri[feature].apply(lambda x: fuzzy_classification(x, min_val, median_val, max_val))


df_fuzzy = pd.concat([fuzzy_df],axis=1)

fuzzy_columns = [col for col in df_fuzzy.columns]

# one-hot encoding
df_fuzzy = pd.get_dummies(df_fuzzy, columns=fuzzy_columns)
df_fuzzy =df_fuzzy.astype(int)
df_fuzzy

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv(r'Customer-Churn-Records.csv')


#print(df_encoded.columns)
y = df['Exited'].astype(int)
#df_encoded = df.astype(float)
#df_encoded
#X=df_encoded.drop(['Churn'], axis=1)

df= df[['CreditScore', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'EstimatedSalary',
       'Exited', 'Satisfaction Score',
       'Point Earned']]

df = pd.DataFrame(df)

# LabelEncoder
label_encoder = LabelEncoder()



df

In [ ]:
#Top 10 patterns


('Age_fuzzy_1', 'NumOfProducts_fuzzy_0') -> Utility: 2012.0
('Age_fuzzy_1', 'CreditScore_fuzzy_1', 'NumOfProducts_fuzzy_0') -> Utility: 1725.0
('Age_fuzzy_1', 'Balance_fuzzy_1', 'NumOfProducts_fuzzy_0') -> Utility: 1713.0
('Age_fuzzy_1', 'IsActiveMember_fuzzy_0', 'NumOfProducts_fuzzy_0') -> Utility: 1650.0
('Age_fuzzy_1', 'Balance_fuzzy_1') -> Utility: 1496.0
('Age_fuzzy_1', 'CreditScore_fuzzy_1') -> Utility: 1487.0
('Age_fuzzy_1', 'IsActiveMember_fuzzy_0') -> Utility: 1469.0
('Age_fuzzy_1', 'Balance_fuzzy_1', 'CreditScore_fuzzy_1', 'NumOfProducts_fuzzy_0') -> Utility: 1438.0
('Age_fuzzy_1', 'CreditScore_fuzzy_1', 'IsActiveMember_fuzzy_0', 'NumOfProducts_fuzzy_0') -> Utility: 1377.0
('Age_fuzzy_1', 'Balance_fuzzy_1', 'IsActiveMember_fuzzy_0', 'NumOfProducts_fuzzy_0') -> Utility: 1366.0

In [ ]:
X = pd.merge(df, df_fuzzy, left_index=True, right_index=True, how='left')
X['fuzzyp1']=X['Age_fuzzy_1']*X['NumOfProducts_fuzzy_0']
X['fuzzyp2']=X['Age_fuzzy_1']*X['CreditScore_fuzzy_1']*X['NumOfProducts_fuzzy_0']
X['fuzzyp3']=X['Age_fuzzy_1']*X['Balance_fuzzy_1']*X['NumOfProducts_fuzzy_0']
X['fuzzyp4']=X['Age_fuzzy_1']*X['NumOfProducts_fuzzy_0']*X['IsActiveMember_fuzzy_0']
X['fuzzyp5']=X['Age_fuzzy_1']*X['Balance_fuzzy_1']
X['fuzzyp6']=X['Age_fuzzy_1']*X['CreditScore_fuzzy_1']
X['fuzzyp7']=X['Age_fuzzy_1']*X['IsActiveMember_fuzzy_0']
X['fuzzyp8']=X['Age_fuzzy_1']*X['Balance_fuzzy_1']*X['CreditScore_fuzzy_1']*X['NumOfProducts_fuzzy_0']
X['fuzzyp9']=X['Age_fuzzy_1']*X['CreditScore_fuzzy_1']*X['IsActiveMember_fuzzy_0']*X['NumOfProducts_fuzzy_0']
X['fuzzyp10']=X['Age_fuzzy_1']*X['Balance_fuzzy_1']*X['IsActiveMember_fuzzy_0']*X['NumOfProducts_fuzzy_0']


#*X['Subscription  Length_fuzzy_1']*X['Tariff Plan_1']
#X=X[X['fuzzyp']==0]
#y=X[X['CHURN','fuzzyp']]
X=X.drop(['CreditScore_fuzzy_0', 'CreditScore_fuzzy_1', 'CreditScore_fuzzy_2',
       'Age_fuzzy_0', 'Age_fuzzy_1', 'Age_fuzzy_2', 'Tenure_fuzzy_0',
       'Tenure_fuzzy_1', 'Tenure_fuzzy_2', 'Balance_fuzzy_0',
       'Balance_fuzzy_1', 'Balance_fuzzy_2', 'NumOfProducts_fuzzy_0',
       'NumOfProducts_fuzzy_1', 'NumOfProducts_fuzzy_2',
       'IsActiveMember_fuzzy_0', 'IsActiveMember_fuzzy_2',
       'EstimatedSalary_fuzzy_0', 'EstimatedSalary_fuzzy_1',
       'EstimatedSalary_fuzzy_2', 'Exited_fuzzy_0',
       'Satisfaction Score_fuzzy_0', 'Satisfaction Score_fuzzy_1',
       'Satisfaction Score_fuzzy_2', 'Point Earned_fuzzy_0',
       'Point Earned_fuzzy_1', 'Point Earned_fuzzy_2','Exited'],axis=1)
X = X.fillna(0)
X

In [ ]:
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, matthews_corrcoef, confusion_matrix, classification_report)
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# 分割數據集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)



# 計算scale_pos_weight
scale_pos_weight = sum(y_train == 0) / sum(y_train == 1)

# 設定參數
param_grid = {
    'objective': ['binary:logistic'],
    'scale_pos_weight': [scale_pos_weight],
    'max_depth': [3,4,5],
    'learning_rate': [0.1,0.01,0.2],
    'n_estimators': [300,400,500],
    'verbosity': [0]  # 使用verbosity取代過時的silent參數
}

model = xgb.XGBClassifier()
#clf = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', verbose=1)
model.fit(X_train, y_train)

#print("Best Parameters:", clf.best_params_)

#best_model = clf.best_estimator_

# 預測
y_pred = model.predict(X_test)

# 評估模型
y_prob = model.predict_proba(X_test)[:,1]


auc_score = roc_auc_score(y_test, y_prob)

# 評估模型
print("AUC:", auc_score)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Matthews Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, matthews_corrcoef, confusion_matrix, classification_report)

from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt



# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Summarize class distribution
print("Before SMOTE: ", dict(zip(*np.unique(y_train, return_counts=True))))

# Apply SMOTE to generate synthetic samples
smote = SMOTE(random_state=1)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Summarize the new class distribution
print("After SMOTE: ", dict(zip(*np.unique(y_resampled, return_counts=True))))

# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the model on the resampled dataset
rf_classifier.fit(X_resampled, y_resampled)

# Predict on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the classifier

y_prob = rf_classifier.predict_proba(X_test)[:, 1]


auc_score = roc_auc_score(y_test, y_prob)

# 評估模型
print("AUC:", auc_score)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Matthews Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))